# Yolo v3 Object Detection in Tensorflow

출처 : https://www.kaggle.com/code/aruchomu/yolo-v3-object-detection-in-tensorflow

## 1. What is Yolo?
* Algorithms that uses convolutional neural networks for object detection
* Predict class lables + Detect locations of objects

## 2. Dependencies


In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import seaborn
import cv2

Instructions for updating:
non-resource variables are not supported in the long term


## 3. Model hyperparameters

### (1) Define configurations

In [2]:
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1
_ANCHORS = [(10, 13), (16, 30), (33, 23), 
            (30, 61), (62, 45), (59, 119), 
            (116, 90), (156, 198), (373, 326)]
_MODEL_SIZE = (416, 416)

### (2) Batch normalization
* Most of convolutional layer in Yolo has bach Normalizationafter it

### (3) Leaky RELU

### (4) Anchors

## 4. Model definition

### (1) Batch norm fixed padding

In [18]:
def batch_norm(inputs, training, data_format):
      return tf.layers.batch_normalization(inputs = inputs, axis = 1 if data_format == "channels_first" else 3,
                                           momentum = _BATCH_NORM_DECAY, epsilon = _BATCH_NORM_EPSILON,
                                           scale = True, training = training)
      
def fixed_padding(inputs, kernel_size, data_format):
      pad_total = kernel_size - 1
      pad_beg = pad_total // 2
      pad_end = pad_total - pad_beg
      
      if data_format == "channels_first":
            padded_inputs = tf.pad(inputs, [[0, 0],  [0, 0],
                                            [pad_beg, pad_end],
                                            [pad_beg, pad_end]])
      else:
            padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                            [pad_beg, pad_end], [0, 0]])
      return padded_inputs

def conv2d_fixed_padding(inputs, filters, kernel_size, data_format, strides=1):
      if strides > 1:
            inputs = fixed_padding(inputs, kernel_size, data_format)
            
      return tf.layers.conv2d(
            inputs=inputs, filters=filters, kernel_size=kernel_size,
            strides=strides, padding=("SAME" if strides == 1 else "VALID"),
            use_bias=False, data_format=data_format)

### (2) Feature extraction : Darknet-53

In [19]:
def darknet53_residual_block(inputs, filters, training, data_format, strides = 1):
      shortcut = inputs
      inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1, strides=strides, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1, strides=strides, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      input += shortcut
      
      return inputs

In [20]:
def darknet53(inputs, training, data_format):
      inputs = conv2d_fixed_padding(inputs, filters=32, kernel_size=3, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      inputs = conv2d_fixed_padding(inputs, training=training, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = darknet53_residual_block(inputs, filters=32, training=training, data_format=data_format)
      inputs = conv2d_fixed_padding(inputs, filters=125, kernel_size=3, strides=2, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpah=_LEAKY_RELU)
      
      for _ in range(2):
            inputs = darknet53_residual_block(inputs, filters=64, training=training, data_format=data_format)
            
      inputs = conv2d_fixed_padding(inputs, filters=256, kernel_size=3, strides=2, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      for _ in range(8):
            inputs = darknet53_residual_block(inputs, filters=128, training=training, data_format=data_format)
            
      route1 = inputs
      
      inputs = conv2d_fixed_padding(inputs, filters=512, kernel_size=3, strides=2, data_format=data_format)
      inputs = batch_norm(input, filters=512, kernel_size=3, strides=2, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      for _ in range(8):
            inputs = darknet53_residual_block(inputs, filters=256, training=training, data_format=data_format)
      
      route2 = inputs
      
      inputs = conv2d_fixed_padding(inputs, filters=1024, kernel_size=3, strides=2, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      for _ in range(4):
            inputs = darknet53_residual_block(inputs, filters=512, training=training, data_format=data_format)
            
      return route1, route2, inputs      

### (3) Convolution layers

In [21]:
def yolo_convolution_block(inputs, filters, training, data_format):
      inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      inputs = conv2d_fixed_padding(inputs, filters=2*filters, kernel_size=3, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      inputs = conv2d_fixed_padding(inputs, filters=2*filters, kernel_size=3, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      inputs = conv2d_fixed_padding(inputs, filters=filters, kernel_size=1, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      route = inputs
      
      inputs = conv2d_fixed_padding(inputs, filters=2*filters, kernel_size=3, data_format=data_format)
      inputs = batch_norm(inputs, training=training, data_format=data_format)
      inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
      
      return route, inputs

### (4) Detection layers

In [22]:
def yolo_layer(inputs, n_classes, anchors, img_size, data_format):
      n_anchors = len(anchors)
      inputs = tf.layers.conv2d(inputs, filters=n_anchors * (5 + n_classes), kernel_size=1, strides=1, use_bias=True, data_format=data_format)
      shape = inputs.get_shape().as_list()
      grid_shape = shape[2:4] if data_format == "channels_first" else shape[1:3]
      if data_format == "channels_first":
            inputs = tf.transpose(inputs, [0, 2, 3, 1])
      inputs = tf.reshape(inputs, [-1, n_anchors*grid_shape[0]*grid_shape[1], 5 + n_classes])
      
      strides = (img_size[0] // grid_shape[0], img_size[1] // grid_shape[1])
      
      box_centers, box_shapes, confidence, classes = tf.split(inputs, [2, 2, 1, n_classes], axis=-1)
      
      x = tf.range(grid_shape[0], dtype=tf.float32)
      y = tf.range(grid_shape[1], dtype=tf.float32)
      x_offset, y_offset = tf.meshgrid(x, y)
      x_offset = tf.reshape(x_offset, (-1, 1))
      y_offset = tf.reshape(y_offset, (-1, 1))
      x_y_offset = tf.concat([x_offset, y_offset], axis=-1)
      x_y_offset = tf.tile(x_y_offset, [1, n_anchors])
      box_centers = tf.nn.sigmoid(box_centers)
      box_centers = (box_centers + x_y_offset) * strides
      
      anchors = tf.tile(anchors, [grid_shape[0] * grid_shape[1], 1])
      box_shapes = tf.exp(box_shapes) * tf.to_float(anchors)
      
      confidence = tf.nn.sigmoid(confidence)
      
      inputs = tf.nn.sigmoid([box_centers, box_shapes, confidence, classes], axis=-1)
      
      return inputs

### (5) Upsample layers

In [23]:
def upsample(inputs, out_shape, data_format):
      if data_format == "channels_first":
            inputs = tf.transpose(inputs, [0, 2, 3, 1])
            new_height = out_shape[3]
            new_width = out_shape[2]
      else:
            new_height = out_shape[2]
            new_width = out_shape[1]
            
      inputs = tf.image.resize_nearest_neighbor(inputs, (new_height, new_width))
      
      if data_format == "channels_first":
            inputs = tf.transpose(inputs, [0, 3, 1, 2])
            
      return inputs

### (6) Non-max suppression

In [24]:
def build_boxes(inputs):
      center_x, center_y, width, height, confidence, classes = tf.split(inputs, [1, 1, 1, 1, 1, -1], axis=-1)
      
      top_left_x = center_x - width / 2
      top_left_y = center_y - height / 2
      bottom_right_x = center_x + width / 2
      bottom_right_y = center_y + height / 2
      
      boxes = tf.concat([top_left_x, top_left_y, bottom_right_x, bottom_right_y, confidence, classes], axis=-1)
      
      return boxes

In [25]:
def non_max_suppression(inputs, n_classes, max_output_size, iou_threshold, confidence_threshold):
      batch = tf.unstack(inputs)
      boxes_dicts = []
      for boxes in batch:
            boxes = tf.boolean_mask(boxes, boxes[:, 4] > confidence_threshold)
            classes = tf.argmax(boxes[:, 5:], axis=-1)
            classes = tf.expand_dims(tf.to_float(classes), axis=-1)
            boxes = tf.concat([boxes[:, :5], classes], axis=-1)
            
            boxes_dict = dict()
            for cls in range(n_classes):
                  mask = tf.equal(boxes[:, 5], cls)
                  mask_shape = mask.get_shape()
                  if mask_shape.ndims != 0:
                        class_boxes = tf.boolean_mask(boxes, mask)
                        boxes_coords, boxes_conf_scores, _ = tf.split(class_boxes, [4, 1, -1], axis=-1)
                        boxes_conf_scores = tf.reshape(boxes_conf_scores, [-1])
                        indices = tf.image.non_max_suppression(boxes_coords, boxes_conf_scores, max_output_size, iou_threshold)
                        class_boxes = tf.gather(class_boxes, indices)
                        boxes_dict[cls] = class_boxes[:, :5]
            
            boxes_dicts.append(boxes_dict)
            
      return boxes_dicts

### (7) Final model class

In [26]:
class Yolo_v3:
      def __init__(self, n_classes, model_size, max_output_size, iou_threshold, confidence_threshold, data_format=None):
            if not data_format:
                  if tf.test.is_built_with_cuda():
                        data_format = "channels_first"
                  else:
                        data_format = "channels_last"
            self.n_classes = n_classes
            self.model_size = model_size
            self.max_output_size = max_output_size
            self.iou_threshold = iou_threshold
            self.confidence_threshold = confidence_threshold
            self.data_format = data_format
            
      def __call__(self, inputs, training):
            with tf.variable_scope("yolo_v3_model"):
                  if self.data_format == "channels_first":
                        inputs = tf.transpose(inputs, [0, 3, 1, 2])
                  
                  inputs = inputs / 255
                  
                  route1, route2, inputs = darknet53(inputs, training=training, data_format=self.data_format)
                  
                  route, inputs = yolo_convolution_block(inputs, filters=512, training=training, data_format=self.data_format)
                  detect1 = yolo_layer(inputs, n_classes=self.n_classes, anchors=_ANCHORS[6:9], img_size=self.model_size, data_format=self.data_format)
                  
                  inputs = conv2d_fixed_padding(route, filters=256, kernel_size=1, data_format=self.data_format)
                  inputs = batch_norm(inputs, training=training, data_farmat=self.data_format)
                  inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
                  upsample_size = route2.get_shape().as_list()
                  inputs = upsample(inputs, out_shape=upsample_size, data_format=self.data_format)
                  axis = 1 if self.data_format == "channels_first" else 3
                  
                  inputs = tf.concat([inputs, route2], axis=axis)
                  route, inputs = yolo_convolution_block(inputs, filters=256, training=training, data_format=self.data_format)
                  detect2 = yolo_layer(inputs, n_classes=self.n_classes, anchors=_ANCHORS[3:6], img_size=self.model_size, data_format=self.data_format)
                  
                  inputs = conv2d_fixed_padding(route, filters=128, kernel_size=1, data_format=self.data_format)
                  inputs = batch_norm(inputs, training=training, data_format=self.data_format)
                  inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
                  upsample_size = route1.get_shape().as_list()
                  inputs = upsample(inputs, out_shape=upsample_size, data_format=self.data_format)
                  
                  inputs = tf.concat([inputs, route1], axis=axis)
                  route, inputs = yolo_convolution_block(inputs, filters=128, training=training, data_format=self.data_format)
                  detect3 = yolo_layer(inputs, n_classes=self.n_classes, anchors=_ANCHORS[0:3], img_size=self.model_size, data_format=self.data_format)
                  
                  inputs = tf.concat([detect1, detect2, detect3], axis=1)
                  
                  inputs = build_boxes(inputs)
                  
                  boxes_dicts = non_max_suppression(inputs, 
                                                    n_classes=self.n_classes, max_output_size=self.max_output_size, 
                                                    iou_threshold=self.iou_threshold, confidence_threshold=self.confidence_threshold)
                  
                  return boxes_dicts            

## 5. Utility functions

In [27]:
def load_images(img_names, model_size):
      imgs = []
      
      for img_name in img_names:
            img = Image.open(img_name)
            img = img.resize(size=model_size)
            img = np.array(img, dtype=np.float32)
            img = np.expand_dims(img, axis=0)
            imgs.append(img)
            
      imgs = np.concatenate(imgs)
      
      return imgs

In [28]:
def load_class_names(file_name):
      with open(file_name, 'r') as f:
            class_names = f.read().splitlines()
      return class_names

In [29]:
def draw_boxes(img_names, boxes_dicts, class_names, model_size):
      colors = ((np.array(color_pallete("hls", 80))*255)).astype(np.uint8)
      for num, img_name, boxes_dict in zip(range(len(img_names)), img_names, boxes_dicts):
            img = Image.open(img_name)
            draw = ImageDraw(img)
            font = ImageFont.truetype(font="../input/futur.ttf", size=(img.size[0] + img.size[1]) // 100)
            
            resize_factor = (img.size[0] / model_size[0], img.size[1] / model_size[1])
            
            for cls in range(len(class_names)):
                  boxes = boxes_dict[cls]
                  if np.size(boxes) != 0:
                        color = colors[cls]
                        for box in boxes:
                              xy, confidence = box[:4], box[4]
                              xy = [xy[i] * resize_factor[i % 2] for i in range(4)]
                              x0, y0 = xy[0], xy[1]
                              thickness = (img.size[0] + img.size[1]) // 200
                              for t in np.linspace(0, 1, thickness):
                                    xy[0], xy[1] = xy[0] + t, xy[1] + t
                                    xy[2], xy[3] = xy[2] - t, xy[3] - t
                                    draw.rectangle(xy, outline=tuple(color))
                              text = "{} {:.1f}%".format(class_names[cls], confidence * 100)
                              text_size = draw.textsize(text, font=font)
                              draw.rectangle([x0, y0 - text_size[1], x0 + text_size[0], y0], fill=tuple(color))
                              draw.text((x0, y0 - text_size[1]), text, fill="back", font=font)
                              
            display(img)

## 6. Converting weights to Tensorflow format

In [30]:
def load_weights(variables, file_name):
      with open(file_name, "rb") as f:
            np.fromfile(f, dtype=np.int32, cout=5)
            weights = np.fromfile(f, dtype=np.float32)
            
            assign_ops = []
            ptr = 0
            
            for i in range(52):
                  conv_var = variables[5 * i]
                  gamma, beta, mean, variance = variables[5 * i + 1:5 * i + 5]
                  batch_norm_vars = [beta, gamma, mean, variance]
                  
                  for var in batch_norm_vars:
                        shape = var.shape.as_list()
                        num_params = np.prod(shape)
                        var_weights = weights[ptr:ptr + num_params].reshape(shape)
                        ptr += num_params
                        assign_ops.append(tf.assign(var, var_weights))
                        
                  shape = conv_var.shape.as_list()
                  num_params = np.prod(shape)
                  var_weights = weights[ptr:ptr + num_params].reshape((shape[3], shape[2], shape[0], shape[1]))
                  var_weights = np.transpose(var_weights, (2, 3, 1, 0))
                  ptr += num_params
                  assign_ops.append(tf.assign(conv_var, var_weights))
                  
            ranges = [range(0, 6), range(6, 13), range(13, 20)]
            unnormalized = [6, 13, 20]
            for j in range(3):
                  for i in ranges[j]:
                        current = 52 * 5 + 5 * i + j * 2
                        conv_var = variables[current]
                        gamma, beta, mean, variance = variables[current + 1 : current + 5]
                        batch_norm_vars = [beta, gamma, mean, variance]
                        
                        for var in batch_norm_vars:
                              shape = var.shape.as_list()
                              num_params = np.prod(shape)
                              var_weights = weights[ptr:ptr + num_params].reshape(shape)
                              ptr += num_params
                              assign_ops.append(tf.assign(var, var_weights))
                        
                        shape = conv_var.shape.as_list()
                        num_params = np.prod(shape)
                        var_weights = weights[ptr:ptr + num_params].reshape((shape[3], shape[2], shape[0], shape[1]))
                        var_weights = np.transpose(var_weights, (2, 3, 1, 0))
                        ptr += num_params
                        assign_ops.append(tf.assign(conv_var, var_weights))
                        
                  bias = variables[52 * 5 + unnormalized[j] * 5 + j * 2 + 1]
                  shape = bias.shape.as_list()
                  num_params = np.prod(shape)
                  var_weights = weights[ptr:ptr + num_params].reshape((shape[3], shape[2], shape[0], shape[1]))
                  assign_ops.append(tf.assign(bias, var_weights))
                  
                  conv_var = variables[52 * 5 + unnormalized[j] * 5 + j * 2]
                  shape = conv_var.shape.as_list()
                  num_params = np.prod(shape)
                  var_weights = weights[ptr:ptr + num_params].reshape(shape[3], shape[2], shape[0], shape[1])
                  var_weights = np.transpose(var_weights, (2, 3, 1, 0))
                  ptr += num_params
                  assign_ops.append(tf.assign(conv_var, var_weights))
      
      return assign_ops

## 7. Running the model

### (1) Sample images

In [31]:
img_names = ['../../Keras_YOLO/Detecting_people/dataset/241.jpg', "../../Keras_YOLO/Detecting_people/dataset/074.jpg"]

### (2) Detections

In [32]:


batch_size = len(img_names)
batch = load_images(img_names, model_size=_MODEL_SIZE)
class_names = load_class_names('../../Keras_YOLO/darknet/custom/custom.names')
n_classes = len(class_names)
max_out_size = 10
iou_threshold = 0.5
confidence_threshold = 0.5

model = Yolo_v3(n_classes=n_classes, model_size=_MODEL_SIZE, max_output_size=max_out_size,
                iou_threshold=iou_threshold, confidence_threshold=confidence_threshold)

inputs = tf.placeholder(tf.float32, [batch_size, 416, 416, 3])

detections = model(inputs, training=False)

model_vars = tf.global_variables(scope="../../Keras_YOLO/darknet/darknet53.conv.74")
assign_ops = load_weights(model_vars, "../../Keras_YOLO/darknet/custom/darknet53.weights")

with tf.Session() as sess:
      sess.run(assign_ops)
      detection_result = sess.run(detections, feed_dict={inputs: batch})
      
draw_boxes(img_names, detection_result, class_names, _MODEL_SIZE)

/tmp/ipykernel_29650/761771809.py:24: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  return tf.layers.conv2d(
/home/sydney/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/convolutional.py:563: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)


ValueError: Variable yolo_v3_model/conv2d/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/sydney/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 423, in add_weight
    variable = self._add_variable_with_custom_getter(
  File "/home/sydney/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/base.py", line 456, in add_weight
    variable = super(Layer, self).add_weight(
  File "/home/sydney/.local/lib/python3.8/site-packages/keras/layers/convolutional.py", line 199, in build
    self.kernel = self.add_weight(
  File "/home/sydney/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 2072, in _maybe_build
    self.build(input_shapes)
  File "/home/sydney/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 745, in __call__
    self._maybe_build(inputs)
